In [1]:
import bisect
import numpy as np

In [2]:
x_vals = np.array([236999.50, 246999.50, 256999.50, 266999.50, 276999.50, 286999.50, 
                   296999.50, 306999.50, 316999.50, 326999.50, 336999.50, 346999.50, 356999.50])

y_vals = np.array([157999.50, 167999.50, 177999.50, 187999.50, 197000.00])

f_vals = np.array([
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [1.49, 0.00, 0.00, 0.00, 0.00],
    [2.32, 0.00, 0.00, 0.00, 0.00],
    [3.09, 0.08, 0.00, 0.00, 0.00],
    [3.88, 1.60, 0.00, 0.00, 0.00],
    [4.43, 2.34, 1.03, 0.00, 0.00],
    [4.37, 3.03, 1.77, 0.00, 0.00],
    [4.62, 3.05, 1.83, 0.53, 0.00],
    [6.17, 4.27, 3.04, 1.83, 0.56],
    [6.68, 4.84, 3.69, 2.47, 1.21],
    [7.10, 5.37, 4.25, 3.07, 1.85],
    [7.61, 5.80, 4.79, 3.64, 2.46]
])

def bilinear_single(x, y):

    x = float(np.clip(x, x_vals[0], x_vals[-1]))
    y = float(np.clip(y, y_vals[0], y_vals[-1]))
    
    # Find indices i, j so that x_vals[i] <= x <= x_vals[i+1], same for y
    i = bisect.bisect_right(x_vals, x) - 1
    j = bisect.bisect_right(y_vals, y) - 1
    i = min(max(i, 0), len(x_vals) - 2)
    j = min(max(j, 0), len(y_vals) - 2)
    
    x0, x1 = x_vals[i], x_vals[i+1]
    y0, y1 = y_vals[j], y_vals[j+1]
    
    f00 = f_vals[i,   j]
    f10 = f_vals[i+1, j]
    f01 = f_vals[i,   j+1]
    f11 = f_vals[i+1, j+1]
    
    if x1 == x0 or y1 == y0:
        return float(f00)
    
    tx = (x - x0) / (x1 - x0)
    ty = (y - y0) / (y1 - y0)
    
    fxy = (f00 * (1 - tx) * (1 - ty) +
           f10 * tx       * (1 - ty) +
           f01 * (1 - tx) * ty       +
           f11 * tx       * ty)
    return float(fxy)

def f_bilinear(x, y):
    x_arr = np.asarray(x)
    y_arr = np.asarray(y)
    if x_arr.shape != y_arr.shape:
        # broadcast to common shape
        X, Y = np.broadcast_arrays(x_arr, y_arr)
    else:
        X, Y = x_arr, y_arr
    
    out = np.empty_like(X, dtype=float)
    it = np.nditer([X, Y, out], flags=['multi_index'], op_flags=[['readonly'], ['readonly'], ['writeonly']])
    for xv, yv, zv in it:
        zv[...] = bilinear_single(float(xv), float(yv))
    return out

In [3]:
print(f_bilinear(356999.50, 157999.50))

7.61


In [4]:
B0 = 21.3494688
B1 = 8.89697237e-05
B2 = -3.60779714e-04
B3 = -1.23505009e-09
B4 = 2.90509491e-10
B5 = 1.80500810e-09

def f_quadratic(x, y, clip=True):
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    z = (B0 + B1*x + B2*y + B3*x*y + B4*x*x + B5*y*y)
    if clip:
        z = np.maximum(0.0, z)
    return z

In [8]:
print(f_quadratic(236999.50, 187999.50))

0.0
